In [1]:
import mesa
print(mesa.__version__)

3.1.1


In [2]:
# Data visualization tools.
import seaborn as sns

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd
from mesa.datacollection import DataCollector

#For Dice control
import random

In [3]:
#Dictionary
snakes = {16: 6, 47: 26, 49: 11, 62: 19, 64: 60, 87: 24, 93: 73, 95: 75, 99: 78}
ladders = {2: 38, 7: 14, 8: 31, 15: 26, 28: 84, 36: 44, 51: 67, 71: 91, 78: 98, 87: 94}

In [4]:
class PlayerAgent(mesa.Agent):
    def __init__(self, model):
        super().__init__(model)
        self.position = 1  # Start at position 1 (bottom-left of the board)
        self.grid_pos = (0, 0)  # Start at (0, 0)
        self.dice_value = 0  # Initialize dice value

    def roll_dice(self):
        """Roll a dice and update the dice_value attribute."""
        self.dice_value = random.randint(1, 6)
        return self.dice_value

    def calculate_new_position(self, new_position):
        """Convert board position to grid coordinates."""
        row = (new_position - 1) // 10
        col = (new_position - 1) % 10
        if row % 2 == 1:  # Reverse direction for odd rows
            col = 9 - col
        return (col, row)

    def move(self):
        """Move the player based on dice roll and handle snakes/ladders."""
        # Roll the dice
        dice_roll = self.roll_dice()
        
        # Calculate new position
        new_position = self.position + dice_roll
        if new_position > 100:
            new_position = self.position  # Stay in the same position if roll exceeds 100

        # Check for snakes or ladders
        if new_position in self.model.snakes:
            print(f"Player {self.unique_id} encountered a snake at {new_position}! Sliding down...")
            new_position = self.model.snakes[new_position]
        elif new_position in self.model.ladders:
            print(f"Player {self.unique_id} climbed a ladder at {new_position}!")
            new_position = self.model.ladders[new_position]

        # Update position and grid
        self.position = new_position
        self.grid_pos = self.calculate_new_position(new_position)

        # Move agent in the grid
        self.model.grid.move_agent(self, self.grid_pos)
        print(f"Player {self.unique_id} rolled {dice_roll} and moved to position {self.position} ({self.grid_pos})")

    def step(self):
        """Define the player's actions for each step."""
        if self.model.winner is None:  # Only proceed if there's no winner yet
            self.move()
            if self.position == 100:
                print(f"Player {self.unique_id} wins!")
                self.model.winner = self.unique_id
                self.model.running = False  # End the simulation

In [5]:
class SnakeAndLaddersModel(mesa.Model):
    def __init__(self, num_players):
        super().__init__()
        self.num_players = num_players
        self.grid = mesa.space.MultiGrid(10, 10, torus=False)  # 10x10 board
        self.snakes = {16: 6, 47: 26, 49: 11, 62: 19, 64: 60, 87: 24, 93: 73, 95: 75, 99: 78}
        self.ladders = {2: 38, 7: 14, 8: 31, 15: 26, 28: 84, 36: 44, 51: 67, 71: 91, 78: 98, 87: 94}
        self.winner = None

        self.player_agents = []  # Custom storage, not overwriting model.agents

        # Add players to the model
        for _ in range(self.num_players):
            player = PlayerAgent(self)
            self.player_agents.append(player)
            self.grid.place_agent(player, (0, 0))  # Place agent at the start
            print(f"Player {player.unique_id} created")  # Verify unique IDs
            
        # Initialize DataCollector
        self.datacollector = DataCollector(
            model_reporters={},
            agent_reporters={
                "Position": lambda agent: agent.position,
                "Dice Value": lambda agent: agent.dice_value
            }
        )

    def step(self):
        """Advance the model by one step."""
        random.shuffle(self.player_agents)  # Shuffle players manually
        for agent in self.player_agents:
            agent.step()
        # Collect data at the end of each step
        self.datacollector.collect(self)

In [6]:
# Create initial model instance with 2 players
model1 = SnakeAndLaddersModel(num_players=2)

# Run the model until a player reaches the finish
print("Starting Simulation...\n")

step_count = 0  # Counter to track the number of steps
while model1.winner is None:  # Continue until there is a winner
    step_count += 1
    print(f"\n--- Step {step_count} ---")
    model1.step()

# Announce the winner
print("\nSimulation Complete.")
print(f"Player {model1.winner} has won the game in {step_count} steps!")

Player 1 created
Player 2 created
Starting Simulation...


--- Step 1 ---
Player 1 rolled 5 and moved to position 6 ((5, 0))
Player 2 climbed a ladder at 7!
Player 2 rolled 6 and moved to position 14 ((6, 1))

--- Step 2 ---
Player 1 rolled 6 and moved to position 12 ((8, 1))
Player 2 encountered a snake at 16! Sliding down...
Player 2 rolled 2 and moved to position 6 ((5, 0))

--- Step 3 ---
Player 2 rolled 3 and moved to position 9 ((8, 0))
Player 1 rolled 1 and moved to position 13 ((7, 1))

--- Step 4 ---
Player 1 encountered a snake at 16! Sliding down...
Player 1 rolled 3 and moved to position 6 ((5, 0))
Player 2 rolled 1 and moved to position 10 ((9, 0))

--- Step 5 ---
Player 1 rolled 3 and moved to position 9 ((8, 0))
Player 2 rolled 1 and moved to position 11 ((9, 1))

--- Step 6 ---
Player 2 climbed a ladder at 15!
Player 2 rolled 4 and moved to position 26 ((5, 2))
Player 1 rolled 2 and moved to position 11 ((9, 1))

--- Step 7 ---
Player 1 rolled 3 and moved to position 14

In [14]:
# After simulation
df = model1.datacollector.get_agent_vars_dataframe()
print(df)

              Position  Dice Value
Step AgentID                      
1    1               6           5
     2              14           6
2    1              12           6
     2               6           2
3    1              13           1
     2               9           3
4    1               6           3
     2              10           1
5    1               9           3
     2              11           1
6    1              11           2
     2              26           4
7    1              14           3
     2              29           3
8    1              19           5
     2              30           1
9    1              20           1
     2              35           5
10   1              21           1
     2              39           4
11   1              25           4
     2              42           3
12   1              29           4
     2              26           5
13   1              34           5
     2              32           6
14   1              